In [73]:
from pathlib import Path
import re
import spacy # сначала пробовала NLTK, но в английском языке плохо лемматизируются формы глаголов
import collections
import pandas as pd

In [74]:
!wget https://raw.githubusercontent.com/ov-tokareva/CL-Project/main/An_Introduction_to_Geology.txt
!wget https://raw.githubusercontent.com/ov-tokareva/CL-Project/main/Lumen_Learning_Geology.txt
!wget https://raw.githubusercontent.com/ov-tokareva/CL-Project/main/Physical_Geology_2nd_Ed.txt

--2023-02-18 05:41:04--  https://raw.githubusercontent.com/ov-tokareva/CL-Project/main/An_Introduction_to_Geology.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1092338 (1.0M) [text/plain]
Saving to: ‘An_Introduction_to_Geology.txt’

An_Introduction_to_ 100%[===================>]   1.04M  --.-KB/s    in 0.03s   

2023-02-18 05:41:04 (31.3 MB/s) - ‘An_Introduction_to_Geology.txt’ saved [1092338/1092338]

--2023-02-18 05:41:04--  https://raw.githubusercontent.com/ov-tokareva/CL-Project/main/Lumen_Learning_Geology.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent

In [75]:
text0, text1, text2 = Path('An_Introduction_to_Geology.txt').read_text(), Path('Lumen_Learning_Geology.txt').read_text(), Path('Physical_Geology_2nd_Ed.txt').read_text()
text = " ".join([text0, text1, text2])
print(f'Описание сырых данных\n\nДлина объединенного документа: {len(text)} знаков\n\
Приблизительное общее количество слов: {len(text.split(" "))}')

Описание сырых данных

Длина объединенного документа: 3334187 знаков
Приблизительное общее количество слов: 516254


In [76]:
# замена всех символов, кроме букв, апострофов и дефисов, на пробел
clean_text1 = re.sub("[^a-zA-Z’-]", " ", text)

# проверка результата
clean_text1[7500:8000]

'it together like pieces of a jigsaw puzzle  People noticed the similarities in the coastlines of South America and Africa on the first world maps  and some suggested the continents had been ripped apart  Antonio Snider-Pellegrini did preliminary work on continental separation and matching fossils in       The shape of the continents is different than what is seen by just coastlines  Map of world elevations  Note the light blue  which are continental shelves flooded by shallow ocean water  These '

In [77]:
# удаление лишних пробелов
clean_text = re.sub("\s+", " ", clean_text1)

# проверка результата
clean_text[7500:8000]

' coastlines Map of world elevations Note the light blue which are continental shelves flooded by shallow ocean water These show the true shapes of the continents What Wegener did differently was synthesize a large amount of data in one place He used true edges of the continents based on the shapes of the continental shelves This resulted in a better fit than previous efforts that traced the existing coastlines There are four different fossil organisms that connect South America Africa India Anta'

In [78]:
# добавление собственных стоп-слов
!wget https://raw.githubusercontent.com/ov-tokareva/CL-Project/main/mystopwords.txt
new_stopwords = Path('mystopwords.txt').read_text()
new_stopwords = new_stopwords.split('\n')
new_stopwords

--2023-02-18 05:41:05--  https://raw.githubusercontent.com/ov-tokareva/CL-Project/main/mystopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 150 [text/plain]
Saving to: ‘mystopwords.txt’

mystopwords.txt     100%[===================>]     150  --.-KB/s    in 0s      

2023-02-18 05:41:06 (11.0 MB/s) - ‘mystopwords.txt’ saved [150/150]



['figure',
 'chapter',
 'area',
 'large',
 'show',
 'year',
 'cc',
 'cause',
 'example',
 'call',
 'like',
 'different',
 'create',
 'license',
 'know',
 'answer',
 'correct',
 'incorrect',
 'exercise',
 'km',
 '']

In [79]:
en = spacy.load('en_core_web_sm')
spacy_sw = en.Defaults.stop_words
print('Spacy: ', type(spacy.load('en_core_web_sm').Defaults.stop_words))

Spacy:  <class 'set'>


In [80]:
sw_set = set(en.Defaults.stop_words)
for elem in new_stopwords:
  spacy_sw.add(elem)
print(f"Образец дополненного множества\n", sw_set)

Образец дополненного множества
 {'front', 'meanwhile', 'whereas', 'license', 'became', 'area', 'i', 'first', 'move', 'sometimes', 'formerly', 'beyond', 'against', 'his', 'quite', 'of', 'take', 'know', 'would', 'not', 'sometime', 'anything', 'various', 'therefore', 'toward', 'e', 'what', 'hereafter', 'few', 'hers', 'p', 'has', 'put', 'too', 're', 'upon', 'seemed', 'used', "n't", 'another', 'had', 'seeming', 'indeed', 'somehow', 'thru', 'might', 'some', 'still', 'yourselves', 'latterly', 'somewhere', 'example', 'due', 'anyone', 'made', '’m', '’re', 'really', 'whose', 'such', 'cannot', 'though', 'top', 'ourselves', 'he', 'figure', 'we', 'regarding', '‘m', 'yet', 'm', 'exercise', 'by', 'nevertheless', '‘ll', 'anyway', 'well', "'re", 'keep', 'thereby', 'chapter', 'until', '’ll', 'b', 'along', 'that', 'while', 'seem', 'my', 'their', 'hence', 'or', 'just', 'two', '’s', 'whither', 'twelve', 'mine', 'herself', '‘ve', 'hundred', 'together', 'create', "'d", 'answer', 'when', 'almost', 'using', 's

In [81]:
# лемматизация и токенизация
nlp = spacy.load('en_core_web_sm')
nlp.max_length = len(text) # строка нужна для преодоления ограничения в 1,000,000 символов
doc = nlp(clean_text)

In [82]:
lemmatized_tokens = [token.lemma_.lower() for token in doc if not token.is_stop]
print(lemmatized_tokens[5000:6000])

['live', 'complete', 'darkness', 'ocean', 'floor', 'deprive', 'oxygen', 'sunlight', 'use', 'geothermal', 'energy', 'provide', 'vent', 'process', 'call', 'bacterial', 'chemosynthesis', 'feed', 'sulfur', 'compound', 'discovery', 'scientist', 'believe', 'life', 'earth', 'exist', 'photosynthesis', 'process', 'require', 'sunlight', 'scientist', 'suggest', 'type', 'environment', 'origin', 'life', 'earth', 'extraterrestrial', 'life', 'galaxy', 'jupiter', 'moon', 'europa', 'quiz', 'check', 'comprehension', 'section', 'click', 'directly', 'button', 'bar', 'feature', 'find', 'rift', 'zone', 'high', 'mountain', 'range', 'continent', 'tsunamis', 'earthquake', 'volcanic', 'arc', 'trenches', 'grabens', 'faults', 'rift', 'angle', 'rifts', 'know', 'normal', 'faulting', 'angle', 'form', 'fail', 'rift', 'arm', 'valley', 'call', 'graben', 'rifts', 'know', 'normal', 'faulting', 'angle', 'form', 'fail', 'rift', 'arm', 'valley', 'call', 'graben', 'midocean', 'ridge', 'earth', 'good', 'current', 'active', 'r

In [83]:
from collections import Counter
glossary = Counter(lemmatized_tokens)
freq_glossary = glossary.most_common(500)
freq_glossary[:10]

[('-', 5197),
 ('rock', 4267),
 ('water', 2362),
 ('mineral', 2012),
 ('form', 1779),
 ('earth', 1751),
 ('plate', 1346),
 ('surface', 1283),
 ('flow', 1132),
 ('earthquake', 1099)]

In [84]:
# визуализация результатов
df = pd.DataFrame(freq_glossary, columns =['word', 'frequency'])
df.style

,word,frequency
0,-,5197
1,rock,4267
2,water,2362
3,mineral,2012
4,form,1779
5,earth,1751
6,plate,1346
7,surface,1283
8,flow,1132
9,earthquake,1099


In [85]:
df.to_excel("geology_frequent_words.xlsx")